# Dependencies:
Tensorflow (using 1.5 because newer versions broken on mac for some reason)

`pip3 install tensorflow==1.5`

Keras

`pip3 install keras`

OpenCV

`pip3 install opencv-python`

ImageAI

`pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl`

Pillow, Scipy, numpy, matplotlib
`pip3 install pillow scipy numpy matplotlib`

In [98]:
# Before beginning, download TinyImageNet to the directory from: https://tiny-imagenet.herokuapp.com/
zip_file_url = 'http://cs231n.stanford.edu/tiny-imagenet-200.zip'
import requests, zipfile, io
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [78]:
train_dir = os.path.join(os.getcwd(), 'tiny-imagenet-200', 'train')
#get all the subdirectories from train/, 
training_wnids = [x for x in os.listdir(train_dir)]

lines = [line.rstrip('\n').split('\t') for line in open('tiny-imagenet-200/words.txt')]
wnids_to_words = {line[0]:line[1] for line in lines }

train_bboxes = {} #{wnid: {image_name: bboxes}
for id in training_wnids:
    path = os.path.join(train_dir, id, id+"_boxes.txt")
    lines = [line.rstrip('\n').split('\t') for line in open(path)]
    train_bboxes[id] = { line[0]:np.array(line[1:], dtype=np.int) for line in lines }


In [71]:
train_bboxes[training_wnids[0]]

{'n02795169_0.JPEG': array([19,  0, 60, 63]),
 'n02795169_1.JPEG': array([ 0,  4, 63, 63]),
 'n02795169_2.JPEG': array([22, 49, 63, 63]),
 'n02795169_3.JPEG': array([10,  0, 55, 62]),
 'n02795169_4.JPEG': array([ 6,  0, 57, 63]),
 'n02795169_5.JPEG': array([ 6,  4, 49, 63]),
 'n02795169_6.JPEG': array([ 5, 33, 31, 63]),
 'n02795169_7.JPEG': array([11, 12, 55, 49]),
 'n02795169_8.JPEG': array([13,  7, 45, 55]),
 'n02795169_9.JPEG': array([ 3, 23, 40, 63]),
 'n02795169_10.JPEG': array([ 1,  0, 63, 63]),
 'n02795169_11.JPEG': array([ 0,  0, 49, 63]),
 'n02795169_12.JPEG': array([ 1, 33, 19, 62]),
 'n02795169_13.JPEG': array([ 0,  0, 60, 63]),
 'n02795169_14.JPEG': array([ 1,  8, 63, 56]),
 'n02795169_15.JPEG': array([ 0,  0, 63, 63]),
 'n02795169_16.JPEG': array([39, 14, 63, 60]),
 'n02795169_17.JPEG': array([37,  5, 63, 56]),
 'n02795169_18.JPEG': array([ 0,  0, 63, 63]),
 'n02795169_19.JPEG': array([11,  1, 55, 45]),
 'n02795169_20.JPEG': array([ 9,  9, 55, 63]),
 'n02795169_21.JPEG': a

In [74]:
wnids_to_words

{'n00001740': 'entity',
 'n00001930': 'physical entity',
 'n00002137': 'abstraction, abstract entity',
 'n00002452': 'thing',
 'n00002684': 'object, physical object',
 'n00003553': 'whole, unit',
 'n00003993': 'congener',
 'n00004258': 'living thing, animate thing',
 'n00004475': 'organism, being',
 'n00005787': 'benthos',
 'n00005930': 'dwarf',
 'n00006024': 'heterotroph',
 'n00006150': 'parent',
 'n00006269': 'life',
 'n00006400': 'biont',
 'n00006484': 'cell',
 'n00007347': 'causal agent, cause, causal agency',
 'n00007846': 'person, individual, someone, somebody, mortal, soul',
 'n00015388': 'animal, animate being, beast, brute, creature, fauna',
 'n00017222': 'plant, flora, plant life',
 'n00019046': 'native',
 'n00019128': 'natural object',
 'n00019613': 'substance',
 'n00020090': 'substance',
 'n00020827': 'matter',
 'n00021265': 'food, nutrient',
 'n00021734': 'nutrient',
 'n00021939': 'artifact, artefact',
 'n00022903': 'article',
 'n00023100': 'psychological feature',
 'n0002

False

barrel, cask
backpack, back pack, knapsack, packsack, rucksack, haversack
espresso
bullet train, bullet
European fire salamander, Salamandra salamandra
albatross, mollymawk
barn
snorkel
beach wagon, station wagon, wagon, estate car, beach waggon, station waggon, waggon
bikini, two-piece
scorpion
jellyfish
lawn mower, mower
Egyptian cat
tarantula
comic book
oboe, hautboy, hautbois
orangutan, orang, orangutang, Pongo pygmaeus
coral reef
computer keyboard, keypad
goldfish, Carassius auratus
syringe
fur coat
picket fence, paling
confectionery, confectionary, candy store
swimming trunks, bathing trunks
sock
bucket, pail
ice cream, icecream
cash machine, cash dispenser, automated teller machine, automatic teller machine, automated teller, automatic teller, ATM
sombrero
plate
cannon
monarch, monarch butterfly, milkweed butterfly, Danaus plexippus
gasmask, respirator, gas helmet
organ, pipe organ
ox
boa constrictor, Constrictor constrictor
koala, koala bear, kangaroo bear, native bear, Phascol

In [85]:
from imageai.Detection import ObjectDetection
import os

execution_path = os.getcwd()

test_id = 'n07920052'
test_img_num = 1
test_img_path = os.path.join(train_dir, test_id, 'images', test_id + '_' + str(test_img_num) + '.JPEG')

detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath( os.path.join(execution_path , "yolo.h5"))
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , test_img_path), output_image_path=os.path.join(execution_path , "test.jpg"), minimum_percentage_probability=30)
pred_bbox = None
for eachObject in detections:
    pred_bbox = eachObject["box_points"]
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")
    
    
truth_bbox = train_bboxes[test_id][os.path.basename(test_img_path)]

cup  :  41.81547164916992  :  (10, 5, 55, 60)
--------------------------------


In [86]:
truth_bbox

array([14, 10, 52, 54])

In [81]:
wnids_to_words[test_id]

'espresso'

In [95]:
# Give tuples/lists in form (xmin, ymin, xmax, ymax)

def overlap(a, b):  # returns None if rectangles don't intersect
    
    dx = min(a[2], b[2]) - max(a[0], b[0])
    dy = min(a[3], b[3]) - max(a[1], b[1])
    truth_area = (b[3]-b[1])*(b[2]-b[0])
    pred_area = (a[3]-a[1])*(a[2]-a[0])
#     print(truth_area)
    if (dx>=0) and (dy>=0):
        intersect = truth_area+pred_area-dx*dy
        return dx*dy / intersect
    
    
#     dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
#     dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
#     if (dx>=0) and (dy>=0):
#         return dx*dy

In [96]:
overlap(pred_bbox, truth_bbox)

0.6755555555555556

In [ ]:
# from keras.applications.resnet50 import ResNet50
# from keras.preprocessing import image
# from keras.applications.resnet50 import preprocess_input, decode_predictions
# import numpy as np

# model = ResNet50(weights='imagenet')

# img_path = 'elephant.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# preds = model.predict(x)
# # decode the results into a list of tuples (class, description, probability)
# # (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=3)[0])
# # Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]